In [13]:
import networkx as nx
import numpy as np
from sympy import Point3D, Ray3D, Line3D, Plane
from scipy.spatial.distance import euclidean, sqeuclidean

from lib.positions import coordArr
from lib.geometry import closestPoints, createBoundingBox, testBoundingBox, getBoundingPoint
from lib.neurites import findSEEMNeurites, EPSILON
from lib.graphs import G_Kaiser, G_White, G_Witvliet

In [19]:
neurites = findSEEMNeurites(random=True)
%store neurites_random

Stored 'neurites' (dict)


In [20]:
%store -r neurites_random

def createSEEM_FC_Graph():
  G = nx.read_gexf('data/kaiser/131/compiled/xyzp.gexf')
  G_new= nx.create_empty_copy(G)

  maxDistance = MICRON
  neuron_neurite_Tuples = neurites.items()
  for neuron1, neurite1 in neuron_neurite_Tuples:
    for neuron2, neurite2 in neuron_neurite_Tuples:
      if (neuron1 != neuron2): 
        dist = closestPoints(neurite1,neurite2)[2]
        if (dist <= maxDistance):
          coord1 = getCoordinates(G.nodes[neuron1])
          coord2 = getCoordinates(G.nodes[neuron2])
          distance = euclidean(coord1, coord2)
          G_new.add_edge(neuron1, neuron2, distance=distance)
  print('Number of Edges in Fully Connected SEEM Graph:',G_new.number_of_edges())
  nx.write_gexf(G_new, 'data/kaiser/131/compiled/rand/xyzp_d_SEEM_FC.gexf')
  return G_new

In [21]:
G_SEEM_FC = createSEEM_FC_Graph()
%store G_SEEM_FC_Rand

Number of Edges in Fully Connected SEEM Graph: 11580


UsageError: Unknown variable 'G_SEEM_FC_Rand'


In [22]:
def seemInstance(edgeCount):
  if G_SEEM_FC.number_of_edges() < edgeCount:
    print('ERROR: edge count too high')
    return False

  rng = np.random.default_rng()
  keptEdges = rng.choice(list(G_SEEM_FC.edges().data()), size=edgeCount, replace=False)
  
  G = nx.create_empty_copy(G_SEEM_FC)

  G.add_edges_from(keptEdges)

  return G

def generateInstances(edgeCount, n):
  return [seemInstance(edgeCount) for i in range(n)]

In [23]:
def SEEM(dir, n):
  G = nx.read_gexf(dir)
  edgeCount = G.number_of_edges()
  SEEM_G = generateInstances(edgeCount, n)
  return SEEM_G

Stored 'SEEM_Kaiser' (list)
Stored 'SEEM_White' (list)
Stored 'SEEM_Array_Witvliet' (list)


In [24]:
n = 1000
# Generate n instances of the SEEM graph for Kaiser et al. Graph
SEEM_Kaiser_Rand = SEEM(G_Kaiser, n)
%store SEEM_Kaiser_Rand

# Generate n instances of the SEEM graph for White et al. N2U Graph
SEEM_White_Rand = SEEM(G_White, n)
%store SEEM_White_Rand

# Generate n instances of the SEEM graph for all 9 Witvliet et al. Graphs
SEEM_Array_Witvliet_Rand = [SEEM(G_Witvliet[i], n) for i in range(8)]
%store SEEM_Array_Witvliet_Rand

Stored 'SEEM_Kaiser' (list)
Stored 'SEEM_White' (list)
Stored 'SEEM_Array_Witvliet' (list)
